---
# The CKY Algorithm

A context-free grammar parser using the CYK algorithm (and my notes) as described in [Chapter 17](https://web.stanford.edu/~jurafsky/slp3/17.pdf) of the [Speech and Language Processing textbook](https://web.stanford.edu/~jurafsky/slp3/).

---
## Context-Free Grammar

A context-free grammar $G$ is defined by four parameters

<div class="alert alert-info">

$N$ - a set of non-terminal symbols (or variables)  

$Σ$ - a set of terminal symbols (disjoint from $N$)  

$R$ - a set of rules or productions, each of the form $(A \rightarrow \beta)$,  
where $A$ is a non-terminal, $\beta$ is a string of symbols from the infinite set of strings $(\Sigma \cup N)^*$

$S$ - a designated start symbol and a member of $N$

</div>

Capital letters refer to non-terminals. $S$ is the start symbol. Lowercase Greek letters like $\alpha$ and $\gamma$ are strings drawn from $(\Sigma \cup N)^*$. Lowercase Roman letters like $u$ and $w$ represent the strings of terminals.

Let's take a moment to figure out what $(\Sigma \cup N)^*$ means.

In the context of parsing text, the non-terminals represent phrasal categories (e.g., noun phrase, verb phrase), while the terminals represent actual words or tokens in the text. The terminals are typically preceded by pre-terminals that correspond to the parts of speech of the terminal word. 


```text
Constituency tree for "I prefer a morning flight"

         S
       /   \
     NP     VP
     |    /    \
   Pro  Verb    NP
    |    |      /  \
    I  prefer Det  Nom
               |   /  \
               a  Nom  Noun
                  |      |
                 Noun   flight
                  |
                morning
```

```text
[S[NP[Pro "I"]][VP[Verb "prefer"][NP[Det "a"][Nom[Nom[Noun "flight"]][Noun "morning"]]]]]
```

<div class="alert alert-success">

**Note:** The Kleene star operation, denoted with an asterisk $*$, is applied to a set of strings, and represents the set of all possible strings that can be formed by concatenating zero or more strings from the original set.

For example, suppose we have the set $S = \{a, b\}$. Then 

$$S^* = \{\epsilon, a, b, ab, ba, aab, aba, \dots \}$$

where $\epsilon$ is the empty string.
</div>


---
## Loading the Grammar and Lexicon

In [207]:
import os

In [208]:
GRAMMAR_DIR = "grammar"
LEXICON_DIR = "lexicon"

In [209]:
def read_files(language):
    # get the content of the grammar and lexicon files
    GRAMMAR_FILE = os.path.join(GRAMMAR_DIR, f"{language}.txt")
    LEXICON_FILE = os.path.join(LEXICON_DIR, f"{language}.txt")
    with open(GRAMMAR_FILE, "r") as f: grammar_text = f.read()
    with open(LEXICON_FILE, "r") as f: lexicon_text = f.read()
    grammar_lines = grammar_text.strip().split("\n")
    lexicon_lines = lexicon_text.strip().split("\n")
    return grammar_lines, lexicon_lines

def parse_rules(language):
    # parse the grammar and lexicon rules
    # might as well combine them to make them easier to work with
    grammar_lines, lexicon_lines = read_files(language)
    lines = grammar_lines + lexicon_lines
    
    rules = []

    for line in lines:
        # grammar format: 'A' -> 'B' 'C'
        # lexicon format: 'W' -> 'w1' | 'w2 w3' | 'w4'
        lhs, rhs = line.split("->")

        if "|" in rhs:
            # lexicon
            vocabs = rhs.split("|")
            # convert all vocab to lowercase
            # this will resolve the issue of distinguishing between
            # non-terminals (uppercase) and terminals (lowercase)
            # the letter I or city Houston starts with a capital
            # and will make the algo think they are non-terminals
            rhs = [vocab.strip().lower() for vocab in vocabs]
        else:
            # grammar
            phrases = rhs.split()
            rhs = [phrase.strip() for phrase in phrases]

        rules.append((lhs.strip(), rhs))
    return rules

In [210]:
language = "L0"
grammar = parse_rules(language)

In [211]:
grammar[::4]

[('S', ['NP', 'VP']),
 ('NP', ['Proper-Noun']),
 ('Nominal', ['Nominal', 'PP']),
 ('VP', ['Verb', 'PP']),
 ('Noun', ['book', 'flight', 'meal', 'money']),
 ('Aux', ['does'])]

---
## Chomsky Normal Form

In order for the CYK algorithm to work, we need to convert the grammars to the Chomsky Normal Form (CNF). A CFG is in CNF if it has rules of the form


<div class="alert alert-info">

**1.** $A \rightarrow B \  C$

**2.** $A \rightarrow a$

**3.** no $A \rightarrow \epsilon$ productions ($\epsilon$-free)

</div>

where uppercase letters are non-terminals and lowercase letters are terminals. So the rules can expand to either two non-terminals or a single terminal. This means that Chomsky normal form grammars are binary trees down to the pre-lexical node (POS of word).

We can assume that we're dealing with $\epsilon$-free grammar, so we can ignore condition (3). Then there are three situations that need to be addressed:

<div class="alert alert-success">

**1.**
**Rules that mix terminals with non-terminals on the right-hand side**

These can be normalized by introducing dummy non-terminals that map the original terminal So the production 
$$A \rightarrow B \ c$$  

becomes
$$A \rightarrow B \ C$$
$$C \rightarrow c$$


</div>


<div class="alert alert-success">

**2.** 
**Rules that have a single non-terminal on the right-hand side (unit productions)**

Unit productions can be removed by rewriting the right-hand side of the original rules with the right-hand side of all the non-unit production rules that they ultimately lead to. Formally, if $A \overset{*}{\Rightarrow} B$ by a chain of one or more unit productions and $B \rightarrow \gamma$ is a non-unit production in our grammar, then we add $A \rightarrow \gamma$ for each such rule in the grammar and discard all intervening productions.

For a more concrete example, suppose we have rules
\begin{align*}
S &\rightarrow A \\
A &\rightarrow B \\
B &\rightarrow a \ B \ b \\
B &\rightarrow \epsilon \\
\end{align*}

Then the first two rules are unit productions. We can see that $A$ can derive $B$, and $S$ can derive $B$ through $A$. This means that both $S$ and $A$ are able to derive strings that $B$ can derive, so we add the following rules:
\begin{align*}
S &\rightarrow a \ B \ b \\
S &\rightarrow \epsilon \\
A &\rightarrow a \ B \ b \\
A &\rightarrow \epsilon \\
\end{align*}

and remove the original unit production rules from $S$ and $A$ to create a new grammar
\begin{align*}
S &\rightarrow a \ B \ b \\
S &\rightarrow \epsilon \\
A &\rightarrow a \ B \ b \\
A &\rightarrow \epsilon \\
B &\rightarrow a \ B \ b \\
B &\rightarrow \epsilon \\
\end{align*}

</div>

<div class="alert alert-success">

**3.**
**Rules in which the length of the right-hand side is greater than two**

These can be normalized through the introduction of new non-terminals that spread the longer sequences over several new rules. For example, if we have
$$A \rightarrow B \ C \ \gamma$$

we replace the leftmost pair of non-terminals with a new non-terminal and introduce a new production rule:
$$A \rightarrow X1 \ \gamma$$
$$X1 \rightarrow B \ C$$
</div>





The CNF conversion algorithm runs as follows:

1) copy all conforming rules to the new grammar unchanged
    - a `grammar` is passed into the `to_cnf` function
    
    - the `grammar` is a list of productions of the form (LHS, [RHS_1, RHS_2, ...])

2) convert terminals within rules to dummy non-terminals
    - the results are stored in `new_productions` 

    - if the RHS of a rule has only non-terminals $\left(A \rightarrow B \ C \right)$, it is copied over into `new_productions` 
    
    - if the RHS includes a non-terminal $\left(A \rightarrow b \ C \right)$, a dummy rule $\left(B \rightarrow b \right)$ is created, the RHS of the previous rule is updated $\left(A \rightarrow B \ C \right)$, and both of these new rules are added to `new_productions`

3) make all rules binary and add them to the new grammar
    - for each rule in `new_productions`, we will reduce the size of the RHS to two non-terminals (we only check for productions with RHS greater than two, so the non-terminal-to-terminal mappings and the unit productions will remain unchanged)

    - dummy non-terminals with the name $X$ replace the first two non-terminals in rules with a RHS with more than 2 non-terminals, e.g., $A \rightarrow B \ C \ D$ becomes $A \rightarrow X1 \ D$ and $X1 \rightarrow B \ C$

    - this step is repeated for each rule until the RHS contains only two non-terminals -- we store the new productions in `binary_productions`

4) convert unit productions
    -   at this point, all productions in `binary_productions` will either produce 
        
        1) binary non-terminals: $A \rightarrow B \ C$ 

        2) unit productions: $A \rightarrow B$ 

        3) mappings to terminals $A \rightarrow a$ 

    - we need to remove the unit productions, which are stored in `unit_productions`

    -  sf


In [212]:
def is_terminal(symbol):
    # we assume that terminal nodes are all lowercase
    # and non-terminal nodes start with an uppercase letter
    return symbol[0].islower()

def to_cnf(grammar):
    '''grammar is list of form ('LHS', ['RHS_1', 'RHS_2', ...])'''


    ''' 1) Rules that mix terminals with non-terminals on the right-hand side'''
    # remove terminals from rhs of production by creating non-terminal mappings
    # e.g, turn (A -> b C) into (A -> B C and B -> b)
    # this section does not modify rules that do not contain rhs terminals 
    #  simply copies them over from grammar to new_productions (A -> B) remains (A -> B)
    terminal_mappings = {} # store the non-terminal-to-terminal mappings: B -> b
    new_productions = []   # store the updated production rules in form ('LHS', ['RHS_1', 'RHS_2', ...])

    for lhs, rhs in grammar:
        new_rhs = []
        for symbol in rhs:
            # iterate over the (non-)terminals on the rhs 
            if is_terminal(symbol):
                if symbol not in terminal_mappings:
                    # convert to non-terminal by uppercasing the word
                    non_terminal = symbol.upper()
                    terminal_mappings[symbol] = non_terminal
                    # add the new rule: A -> a
                    new_productions.append((non_terminal, [symbol]))
                new_rhs.append(terminal_mappings[symbol])
            else:
                # if it is a non-terminal, do nothing
                new_rhs.append(symbol)

        new_productions.append((lhs, new_rhs))


    ''' 2) Rules in which the length of the right-hand side is greater than two'''
    # we will binarize productions before removing unit productions
    # this means that we will reduce the size of the RHS from 
    # len(rhs) > 2 to len(rhs) <= 2 
    binary_productions = [] # store the updated production rules
    counter = 1             # counter is used for creating dummy non-terminals

    for lhs, rhs in new_productions:
        while(len(rhs) > 2):
            # create the dummy terminal 
            new_non_terminal = f"X{counter}"
            counter += 1
            # add the dummy rule: X# -> [first two vals on rhs]
            binary_productions.append((new_non_terminal, rhs[:2]))
            # update the rhs: rhs = [X#, everything but first 2 vals on rhs]
            rhs = [new_non_terminal] + rhs[2:]
        # once the rule has been binarized, or if there is no issue, add it
        binary_productions.append((lhs, rhs))


    ''' 3) Rules that mix terminals with non-terminals on the right-hand side'''
    # at this point, we have binary non-terminal productions, unit productions
    # and mappings to terminals -- we need to remove the unit productions
    unit_productions = []
    for prod in binary_productions:
        lhs, rhs = prod
        # rhs has from [rhs_1, rhs_2, ...]
        # check if rhs has single element, and if that element
        # is not a terminal, e.g., rhs = [A] --> rhs[0] = A is not a terminal
        if len(rhs) == 1 and not is_terminal(rhs[0]):
            unit_productions.append(prod)

    while unit_productions:
        # remove unit prods until they are all gone
        # e.g., lhs == A, rhs == [B]
        unit_lhs, unit_rhs = unit_productions.pop()

        # find all of the possible chains from the rhs of a unit 
        # production into a binary rule and store the rhs of the binary rule
        # e.g. 
        #   unit_prod: (A, [B])
        #   bnry_prod: (B, [C, D])
        #   [C,D] is stored because bnry_prod[0] == B == unit_prod[1][0] == B
        chains = []
        for bnry_prod in binary_productions:
            bnry_lhs, bnry_rhs = bnry_prod # lhs = B, rhs = [C, D]
            if bnry_lhs == unit_rhs[0]:    # if B == [B][0] == B
                chains.append(bnry_rhs)    # store [C, D]
        
        for new_rhs in chains:
            if len(new_rhs) == 1 and not is_terminal(new_rhs[0]):
                # might create more unit prods in process of removing them -- add to list to be removed
                # e.g., unit_prod: (A, [B]), bnry_prods: (B, [C]) and (C, [D, E])
                # we first create (A, [C]) (add to list), and then create (A, [D, E])
                unit_productions.append((unit_lhs, new_rhs))
            else:
                # if the result is not a unit prod, then it is complete!
                binary_productions.append((unit_lhs, new_rhs))
        
        # filter the current unit prod from the binary prods list
        binary_productions = [prod for prod in binary_productions if prod != (unit_lhs, unit_rhs)]
        
    return binary_productions

In [213]:
cnf_grammar = to_cnf(grammar)

In [214]:
cnf_grammar

[('S', ['NP', 'VP']),
 ('X1', ['Aux', 'NP']),
 ('S', ['X1', 'VP']),
 ('NP', ['Det', 'Nominal']),
 ('Nominal', ['Nominal', 'Noun']),
 ('Nominal', ['Nominal', 'PP']),
 ('VP', ['Verb', 'NP']),
 ('X2', ['Verb', 'NP']),
 ('VP', ['X2', 'PP']),
 ('VP', ['Verb', 'PP']),
 ('VP', ['VP', 'PP']),
 ('PP', ['Preposition', 'NP']),
 ('THAT', ['that']),
 ('THIS', ['this']),
 ('THE', ['the']),
 ('A', ['a']),
 ('X3', ['THAT', 'THIS']),
 ('X4', ['X3', 'THE']),
 ('Det', ['X4', 'A']),
 ('BOOK', ['book']),
 ('FLIGHT', ['flight']),
 ('MEAL', ['meal']),
 ('MONEY', ['money']),
 ('X5', ['BOOK', 'FLIGHT']),
 ('X6', ['X5', 'MEAL']),
 ('Noun', ['X6', 'MONEY']),
 ('INCLUDE', ['include']),
 ('PREFER', ['prefer']),
 ('X7', ['BOOK', 'INCLUDE']),
 ('Verb', ['X7', 'PREFER']),
 ('I', ['i']),
 ('SHE', ['she']),
 ('ME', ['me']),
 ('X8', ['I', 'SHE']),
 ('Pronoun', ['X8', 'ME']),
 ('HOUSTON', ['houston']),
 ('NWA', ['nwa']),
 ('Proper-Noun', ['HOUSTON', 'NWA']),
 ('DOES', ['does']),
 ('FROM', ['from']),
 ('TO', ['to']),
 ('O

---
## CKY Parser

\begin{align*}
\text{function} & \ \text{CKY-PARSE}(words, grammar) \ \text{returns table} \\
& \\
& \text{for } j \leftarrow 1 \ \text{to} \ \text{LENGTH}(words) \ \text{do} \\
& \quad \text{for all } \{A \mid A \rightarrow words[j] \in grammar\} \\
& \quad \quad \text{table}[j-1,j] \leftarrow \text{table}[j-1,j] \cup A \\
& \quad \text{for } i \leftarrow j-2 \ \text{down to} \ 0 \ \text{do} \\
& \quad \quad \text{for } k \leftarrow i+1 \ \text{to} \ j-1 \ \text{do} \\
& \quad \quad \quad \text{for all } \{A \mid A \rightarrow BC \in grammar \ \text{and} \ B \in \text{table}[i,k] \ \text{and} \ C \in \text{table}[k,j]\} \\
& \quad \quad \quad \quad \text{table}[i,j] \leftarrow \text{table}[i,j] \cup A \\
\end{align*}


\begin{align*}
\text{function} & \ \text{CKY-PARSE}(words, grammar) \ \text{returns table} \\
& \\
& \text{for } j \leftarrow 1 \ \text{to} \ \text{LENGTH}(words) \ \text{do} \\
& \quad \text{for all } \{A \mid A \rightarrow words[j] \in grammar\} \\
& \quad \quad \text{table}[j-1,j] \leftarrow \text{table}[j-1,j] \cup A \\
& \quad \text{for } i \leftarrow j-2 \ \text{down to} \ 0 \ \text{do} \\
& \quad \quad \text{for } k \leftarrow i+1 \ \text{to} \ j-1 \ \text{do} \\
& \quad \quad \quad \text{for all } \{A \mid A \rightarrow BC \in grammar \\
& \quad \quad \quad \quad \text{and} \ B \in \text{table}[i,k] \ \text{and} \ C \in \text{table}[k,j]\} \\
& \quad \quad \quad \quad \text{table}[i,j] \leftarrow \text{table}[i,j] \cup A \\
\end{align*}


In [221]:
def CKY_parse(sentence, grammar):
    table = []
    words = sentence.split()

    # j in [1, len(words)]
    for j in range(1, len(words)+1):
        print("j:", j)

        # i in [0, j-2]
        for i in range(0, (j-2)+1):
            print("i:", i)

            # k in [i+1, j-1]
            for k in range(i+1, (j-1)+1):
                print("k:", k)

    return table

In [224]:
sentence = "Book the flight through Houston"
# table = CKY_parse(sentence, grammar)

2. Convert terminals within rules to dummy non-terminals.
3. Convert unit productions.
4. Make all rules binary and add them to new grammar.

In [217]:
def find_deps(grammar, symbol):
    # get dependencies for the symbol 
    # there should only be one 
    # the symbol will always be the first element of 
    # the rhs of the production rule:
    #   rule = (lhs, rhs) = (lhs, [symbol, ...])
    deps = []
    for lhs, rhs in grammar:
        if symbol == rhs[0]:
            deps.append((lhs, rhs))    
    return deps   



def unroll(grammar, vocab):
    to_change = []
    for lhs, rhs in grammar:
        first_symbol = rhs[0]
        if first_symbol[0] == "X":
            # add the symbol and rule:
            # (symbol, (lhs, rhs)) = (symbol, (lhs, [symbol, ...]))
            to_change.append((first_symbol, (lhs, rhs)))

    to_remove = []
    for symbol, rule in to_change:
        # get the dependencies for the current rule
        deps = find_deps(grammar, symbol)
        # remove the rule and its dependencies from the grammar
        to_remove += [rule] + deps

        # should only be 1 dep...
        for dep in deps:
            # rule: (r_lhs, [symbol, ...])
            # deps: (symbol, d_rhs)
            # final: (r_lhs, [symbol, d_rhs, ...])
            rule_lhs, rule_rhs = rule
            dep_lhs, dep_rhs = dep
            # omit the first element in rule_rhs
            rule = (rule_lhs, [dep_rhs, rule_rhs[1:]])
            
            # add to grammar
            grammar += rule
    
    # return the updated grammar
    return [rule for rule in grammar if rule not in to_remove]

In [218]:
grammar = [('S', ['X7', 'PREFER']),
         ('X7', ['BOOK', 'INCLUDE']),
         ('PREFER', ['prefer']),
         ('INCLUDE', ['include']),
         ('BOOK', ['book'])]

vocab = ["prefer", "include"]

'''
goal:
    S -> BOOK
    S -> PREFER
    S -> INCLUDE
'''


'\ngoal:\n    S -> BOOK\n    S -> PREFER\n    S -> INCLUDE\n'

In [219]:
unroll(grammar, vocab)

[('X7', ['BOOK', 'INCLUDE']),
 ('PREFER', ['prefer']),
 ('INCLUDE', ['include']),
 ('BOOK', ['book']),
 'S',
 [['X7', 'PREFER'], ['PREFER']]]

In [220]:
def get_all_terminal_expansions(symbol, grammar):
    expansions = set()
    for lhs, rhs in grammar:
        if lhs == symbol:
            if is_terminal(rhs[0]):
                expansions.add(rhs[0])
            else:
                expansions |= get_all_terminal_expansions(rhs[0], grammar)
    return expansions

def unroll(grammar, vocab):
    new_productions = []
    for lhs, rhs in grammar:
        if is_terminal(rhs[0]) or len(rhs) == 1:
            new_productions.append((lhs, rhs))
        else:
            terminal_expansions = get_all_terminal_expansions(rhs[0], grammar)
            for term in terminal_expansions:
                new_rule_rhs = [term] + list(rhs[1:])
                new_productions.append((lhs, new_rule_rhs))
    return new_productions

grammar = [('S', ['X7', 'PREFER']),
           ('X7', ['BOOK', 'INCLUDE']),
           ('PREFER', ['prefer']),
           ('INCLUDE', ['include']),
           ('BOOK', ['book'])]

vocab = ["prefer", "include"]

unroll(grammar, vocab)


[('S', ['book', 'PREFER']),
 ('X7', ['book', 'INCLUDE']),
 ('PREFER', ['prefer']),
 ('INCLUDE', ['include']),
 ('BOOK', ['book'])]

True
